## 예제
1. test_csv폴더에 있는 '유통+업종(백화점,+할인점,+슈퍼마켓,+편의점)+카드소비트렌드+데이터_1901_2109'파일을 로드
2. 각 컬럼의 이름을['기준연월', '업종', '성별', '연령대', '총이용건수', '총이용금액']
3. 기준년월을 시계열데이터로 변경
4. 성별 컬럼의 데이터를 1은 남자,2는 여자로 변경, 9는 무관
5. 남,녀 성별을 기준으로 연도별 건당 이용금액의 평균이 몇인지 출력

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
test = pd.read_csv("../test_csv/유통+업종(백화점,+할인점,+슈퍼마켓,+편의점)+카드소비트렌드+데이터_1901_2109.csv",encoding='euc-kr')

In [ ]:
test.columns = ['기준연월', '업종', '성별', '연령대', '총이용건수', '총이용금액']

In [ ]:
# 판다스에서 지수표현 없애기
pd.options.display.float_format = '{:2f}'.format

In [ ]:
test.head(3)

In [ ]:
# 기준년월 데이터를 시계열데이터로 변경

test['기준연월'] = pd.to_datetime(test['기준연월'],format='%Y%m') #시계열 데이터는 일정한 시간 간격으로 순차적으로 기록된 관측 데이터다
                                                             # format뒤에는 기존의 데이터가 가지고있는 시간을 기록해주면된다 여기는 년,월까지
                                                             # 나와있으니까, Y,m 까지 써주는 거고 뒤에 더 나온다면 시,분,초등으로 나간다.
                                                             # 즉 이 함수는 기존 데이터를 시계열데이터로 변경해주는 함수이다. 

In [ ]:
test.info()

In [ ]:
# 인덱스를 기준년월로 설정
test.set_index('기준연월', inplace=True)

In [ ]:
#연령대별 데이터
test['연령대'].value_counts()

In [ ]:
#성별 컬럼에서 1은 남자 2는 연자 9는 무관

#case1
def change(x):
    if x == 1:
        result = '남자'
    elif x == 2:
        result = '여자'
    elif x == 9:
        result = '무관'
    else:
        result = '-'
    return result

test['성별'].apply(change).value_counts

In [ ]:
#case2
test.loc[test['성별'] == 1,'성별'] = '남자'
test.loc[test['성별'] == 2,'성별'] = '여자'
test.loc[test['성별'] == 9,'성별'] = '무관'
test.head(6)

In [ ]:
#case3
test['성별'].apply(lambda x : '남자' if (x==1)else('여자' if(x==2) else'무관'))

In [ ]:
#case4
test['성별'].replace(
    {
    1 : '남자',
    2 : '여자',
    9 : '무관'
    }
)

In [ ]:
#연도 파생변수 생성
test['연도'] = test.index.strftime('%Y')

In [ ]:
test.head()

In [ ]:
# 건당 이용금액 파생변수 생성
# 총이용금액 / 총이용건수
test['연도별 건당 이용금액'] = (test['총이용금액']/test['총이용건수'])
test.head(6) 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import platform

In [ ]:
# 그래프에서 한글 깨짐 방지 - mac인경우
if platform.system() == 'Darwin':
    plt.rc('font',family = 'AppleGothic')
else:
    plt.rc('font',family = 'Malgun Gothic')    
        

In [ ]:
(test[['연도', '성별', '연도별 건당 이용금액']].groupby(['연도','성별']).mean()).plot.bar()

In [ ]:
pivot = pd.pivot_table(
    test,
    index='연도',
    columns='성별',
    aggfunc='mean',
    values='연도별 건당 이용금액'
)

In [ ]:
pivot.plot.bar()

###온라인 카드 매출과 오프라인 카드 매출을 월별로 비교
1. 오프라인 데이터 df를 월별로 그룹화 총이용금액의 합계
  - 인덱스는 기준년월을 시계열로 변경한 데이터로 설정
2. 온라인 데이터 test_online도 월별로 그룹화 총이용금액의 합계
  - 인덱스는 기준년월을 시계열데이터로 변경하여 설정

In [ ]:
group_data = test.groupby('기준연월').sum()['총이용금액']

In [ ]:
test_online = pd.read_csv("../test_csv/온라인+카드소비트렌드+데이터_1901_2109.csv")

In [ ]:
test_online.head(5)

In [ ]:
# 컬럼의 이름을 변경
test_online.columns = ['기준연월','성별','연령대','총이용건수','총이용금액']

In [ ]:
test_online.head(5)

In [ ]:
# 기준연월을 시계열로 변경하고 인덱스로 설정 -> 시계열 데이터는 작성하고 난다음에 꼭 저장을 해줘야 한다. 
# 기준연월을 시계열로 변경하고 인덱스로 설정 -> pd다음에 적고 변수에 저장을 시켜주지 않으면 시계열데이터로 변경한 정보가 저장이 되지 않는다
test_online['기준연월'] = pd.to_datetime(test_online['기준연월'],format='%Y%m')

In [ ]:
test_online.set_index('기준연월',inplace=True)

In [ ]:
#그룹화
group_data2 = test_online.groupby('기준연월').sum()['총이용금액']

In [ ]:
group_data2

In [ ]:
# 그래프로 시각화

# 그래프의 사이즈 변경
plt.figure(figsize= (14, 6))

# grid 추가
plt.grid(True, axis = 'y')

plt.plot(group_data, label = '오프라인')
plt.plot(group_data2, label = '온라인')
plt.title('온라인/오프라인 카드매출 비교')
plt.legend()
plt.show()

In [319]:
test_online['업종'] = '온라인'

In [320]:
test_online.head()

,성별,연령대,총이용건수,총이용금액,업종
기준연월,,,,,
2019-01-01,1,1,1,110685,온라인
2019-01-01,1,10,2333,98108225,온라인
2019-01-01,1,20,272915,13854081656,온라인
2019-01-01,1,30,1320227,82946127640,온라인
2019-01-01,1,40,1372052,95550848337,온라인


In [322]:
total_test = pd.concat([test,test_online],axis=0)

In [323]:
total_test['업종'].value_counts()

편의점     667
백화점     664
슈퍼마켓    663
할인점     654
온라인     651
Name: 업종, dtype: int64